In [ ]:
import numpy as np
import pandas as pd
import warnings
import re  # Example import, if needed for other parts of your code
import csv  # Example import, if needed for other parts of your code
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    ConfusionMatrixDisplay,
)
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from openpyxl import Workbook


Dạ thưa cô, đoạn là Bagging Models với 22 Features và  HyperParameter Tunning

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
# Đường dẫn tới các file CSV
file_paths_basic = [
    r'G:/My Drive/AIL303m/ail2-main/ail2-main/Feature extract/basic/Merged_BC-15_basic_features.csv',
    r'G:/My Drive/AIL303m/ail2-main/ail2-main/Feature extract/basic/Merged_Huong_thom-1_basic_features.csv',
    r'G:/My Drive/AIL303m/ail2-main/ail2-main/Feature extract/basic/Merged_Nep-87_basic_features.csv',
    r'G:/My Drive/AIL303m/ail2-main/ail2-main/Feature extract/basic/Merged_Q-5_modify_basic_features.csv',
    r'G:/My Drive/AIL303m/ail2-main/ail2-main/Feature extract/basic/Merged_Thien_uu-8_basic_features.csv',
    r'G:/My Drive/AIL303m/ail2-main/ail2-main/Feature extract/basic/Merged_Xi-23_features.csv'
]
results = []
# Suppress specific warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
# Biến để tính tổng độ chính xác
total_accuracy = 0
total_precision = 0
total_recall = 0
total_f1 = 0
# Hàm chính để huấn luyện và đánh giá mô hình
for file_path in file_paths_basic:
    name = file_path
    match = re.search(r'Merged_(.*?)_features', file_path)
    if match:
        extracted_string = match.group(1)
    # Đọc dữ liệu từ file CSV
    print(f"Processing file: {file_path}")
    dataset = pd.read_csv(file_path)
    fold_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F-measure': []
    }
    # Tiền xử lý dữ liệu
    X = dataset.drop(columns=['filename', 'label'], axis=1)
    y = dataset['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    k = 5
    # Create KFold object
    kf = KFold(n_splits=k, shuffle=True, random_state=25)
    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    # Parameter grids
    param_grid_svm = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['rbf', 'linear','poly']
    }

    param_grid_knn = {
        'n_neighbors': range(1, 31),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    }

    param_grid_rf = {
        'n_estimators': [100, 200, 500],
        'max_features': [22],
        'max_depth': [4, 6, 8, 10],
        'criterion': ['gini', 'entropy']
    }

    param_grid_ann = {
        'hidden_layer_sizes': [(50,), (100,), (150,), (100, 100)],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant', 'adaptive']
    }

    param_grid_lr = {
        'C': [0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga'],
        'penalty': ['l2', 'l1'],
        'max_iter': [100, 200, 300]
    }

    # Define the base models
    svm_base = SVC()
    knn_base = KNeighborsClassifier()
    rf_base = RandomForestClassifier(random_state=42)
    ann_base = MLPClassifier(max_iter=1000)
    lr_base = LogisticRegression()

    # GridSearchCV for each model
    gs_svm = GridSearchCV(estimator=svm_base, param_grid=param_grid_svm, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_knn = GridSearchCV(estimator=knn_base, param_grid=param_grid_knn, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_rf = GridSearchCV(estimator=rf_base, param_grid=param_grid_rf, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_ann = GridSearchCV(estimator=ann_base, param_grid=param_grid_ann, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_lr = GridSearchCV(estimator=lr_base, param_grid=param_grid_lr, cv=kf, verbose=1, scoring='accuracy',error_score='raise')

    # Fit GridSearchCV to the data
    gs_svm.fit(X_train, y_train)
    gs_knn.fit(X_train, y_train)
    gs_rf.fit(X_train, y_train)
    gs_ann.fit(X_train, y_train)
    gs_lr.fit(X_train, y_train)

    # Get the best estimators
    best_svm = gs_svm.best_estimator_
    best_knn = gs_knn.best_estimator_
    best_rf = gs_rf.best_estimator_
    best_ann = gs_ann.best_estimator_
    best_lr = gs_lr.best_estimator_

    print("Best SVM Estimator:", best_svm)
    print("Best KNN Estimator:", best_knn)
    print("Best Random Forest Estimator:", best_rf)
    print("Best ANN Estimator:", best_ann)
    print("Best Logistic Regression Estimator:", best_lr)
    # Định nghĩa các bộ phân loại Bagging
    bagging_svm = BaggingClassifier(best_svm, n_estimators=5, random_state=42, bootstrap=True, max_features=22)
    bagging_ann = BaggingClassifier(best_ann, n_estimators=5, random_state=42, bootstrap=True, max_features=22)
    bagging_RF = BaggingClassifier(best_rf, n_estimators=5, random_state=42, bootstrap=True, max_features=22)
    bagging_knn = BaggingClassifier(best_knn, n_estimators=5, random_state=42, bootstrap=True, max_features=22)
    bagging_lr = BaggingClassifier(best_lr,n_estimators=5,random_state=42,bootstrap=True,max_features=22)
    # Định nghĩa VotingClassifier với phương pháp hard voting
    voting_model = VotingClassifier(estimators=[
        ('svm', bagging_svm),
        ('rf', bagging_RF),
        ('ann', bagging_ann),
        ('lr', bagging_lr),
        ('knn', bagging_knn)
    ], voting='soft')  # Sử dụng soft voting
    # Huấn luyện Voting Classifier
    voting_model.fit(X_train, y_train)
    # Dự đoán trên tập kiểm tra
    y_pred = voting_model.predict(X_test)

    # Đánh giá độ chính xác của Voting Classifier
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f'Accuracy of Voting Classifier for {extracted_string}: {accuracy:.4f}')
    print(f'Precision of Voting Classifier for {extracted_string}: {precision:.4f}')
    print(f'Recall of Voting Classifier for {extracted_string}: {recall:.4f}')
    print(f'F1-score of Voting Classifier for {extracted_string}: {f1:.4f}')

    # Store metrics for this fold
    fold_metrics['Accuracy'].append(accuracy)
    fold_metrics['Precision'].append(precision)
    fold_metrics['Recall'].append(recall)
    fold_metrics['F-measure'].append(f1)
    # Calculate average metrics for this dataset
    avg_metrics = {
        'Accuracy': np.mean(fold_metrics['Accuracy']),
        'Precision': np.mean(fold_metrics['Precision']),
        'Recall': np.mean(fold_metrics['Recall']),
        'F-measure': np.mean(fold_metrics['F-measure'])
    }
    results.append([
        file_path,
        f"{avg_metrics['Accuracy'] * 100:.2f}".replace('.', ',') ,
        f"{avg_metrics['Precision'] * 100:.2f}".replace('.', ','),
        f"{avg_metrics['Recall'] * 100:.2f}".replace('.', ',') ,
        f"{avg_metrics['F-measure'] * 100:.2f}".replace('.', ',')
    ])

    # Save results to a CSV file
    output_file = r"G:\My Drive\AIL303m\ail2-main\ail2-main\BaggingResult(22_Tunning).csv"

    # Write the results to a CSV file
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure'])
        writer.writerows(results)



36 features Tunning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
# Đường dẫn tới các file CSV
file_paths_basic = [
    r"C:\Users\Acer\Documents\Zalo Received Files\basic36\Merged_BC-15_features_v2.csv",
    r"C:\Users\Acer\Documents\Zalo Received Files\basic36\Merged_Huong_thom-1_features_v2.csv",
    r"C:\Users\Acer\Documents\Zalo Received Files\basic36\Merged_Nep-87_features_v2.csv",
    r"C:\Users\Acer\Documents\Zalo Received Files\basic36\Merged_Q-5_modify_features_v2.csv",
    r"C:\Users\Acer\Documents\Zalo Received Files\basic36\Merged_Thien_uu-8_features_v2.csv",
    r"C:\Users\Acer\Documents\Zalo Received Files\basic36\Merged_Xi-23_features_v2.csv",
]
results = []
# Suppress specific warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
# Biến để tính tổng độ chính xác
total_accuracy = 0
total_precision = 0
total_recall = 0
total_f1 = 0
# Hàm chính để huấn luyện và đánh giá mô hình
for file_path in file_paths_basic:
    name = file_path
    match = re.search(r'Merged_(.*?)_features', file_path)
    if match:
        extracted_string = match.group(1)
    # Đọc dữ liệu từ file CSV
    print(f"Processing file: {file_path}")
    dataset = pd.read_csv(file_path)
    fold_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F-measure': []
    }
    # Tiền xử lý dữ liệu
    X = dataset.drop(columns=['filename', 'label'], axis=1)
    y = dataset['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    k = 5
    # Create KFold object
    kf = KFold(n_splits=k, shuffle=True, random_state=25)
    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    # Parameter grids
    param_grid_svm = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['rbf', 'linear','poly']
    }

    param_grid_knn = {
        'n_neighbors': range(1, 31),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    }

    param_grid_rf = {
        'n_estimators': [100, 200, 500],
        'max_features': [7,8], 
        'max_depth': [4, 6, 8, 10],
        'criterion': ['gini', 'entropy']
    }

    param_grid_ann = {
        'hidden_layer_sizes': [(50,), (100,), (150,), (100, 100)],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant', 'adaptive']
    }

    param_grid_lr = {
        'C': [0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga'],
        'penalty': ['l2', 'l1'],
        'max_iter': [100, 200, 300]
    }

    # Define the base models
    svm_base = SVC()
    knn_base = KNeighborsClassifier()
    rf_base = RandomForestClassifier(random_state=42)
    ann_base = MLPClassifier(max_iter=1000)
    lr_base = LogisticRegression()

    # GridSearchCV for each model
    gs_svm = GridSearchCV(estimator=svm_base, param_grid=param_grid_svm, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_knn = GridSearchCV(estimator=knn_base, param_grid=param_grid_knn, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_rf = GridSearchCV(estimator=rf_base, param_grid=param_grid_rf, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_ann = GridSearchCV(estimator=ann_base, param_grid=param_grid_ann, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_lr = GridSearchCV(estimator=lr_base, param_grid=param_grid_lr, cv=kf, verbose=1, scoring='accuracy',error_score='raise')

    # Get the best estimators
    best_svm = gs_svm.best_estimator_
    best_knn = gs_knn.best_estimator_
    best_rf = gs_rf.best_estimator_
    best_ann = gs_ann.best_estimator_
    best_lr = gs_lr.best_estimator_

    print("Best SVM Estimator:", best_svm)
    print("Best KNN Estimator:", best_knn)
    print("Best Random Forest Estimator:", best_rf)
    print("Best ANN Estimator:", best_ann)
    print("Best Logistic Regression Estimator:", best_lr)
    # Định nghĩa các bộ phân loại Bagging
    bagging_svm = BaggingClassifier(best_svm, n_estimators=i, random_state=42, bootstrap=True, max_features=52)
    bagging_ann = BaggingClassifier(best_ann, n_estimators=i, random_state=42, bootstrap=True, max_features=52)
    bagging_RF = BaggingClassifier(best_rf, n_estimators=i, random_state=42, bootstrap=True, max_features=52)
    bagging_knn = BaggingClassifier(best_knn, n_estimators=i, random_state=42, bootstrap=True, max_features=52)
    bagging_lr = BaggingClassifier(best_lr,n_estimators=i,random_state=42,bootstrap=True,max_features=52)
    # Định nghĩa VotingClassifier với phương pháp hard voting
    voting_model = VotingClassifier(estimators=[
        ('svm', bagging_svm),
        ('rf', bagging_RF),
        ('ann', bagging_ann),
        ('lr', bagging_lr),
        ('knn', bagging_knn)
    ], voting='soft')  # Sử dụng soft voting
    # Huấn luyện Voting Classifier
    voting_model.fit(X_train, y_train).to(device)

    # Dự đoán trên tập kiểm tra
    y_pred = voting_model.predict(X_test)

    # Đánh giá độ chính xác của Voting Classifier
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f'n_estimators= {i}')
    print(f'Accuracy of Voting Classifier for {extracted_string}: {accuracy:.4f}')
    print(f'Precision of Voting Classifier for {extracted_string}: {precision:.4f}')
    print(f'Recall of Voting Classifier for {extracted_string}: {recall:.4f}')
    print(f'F1-score of Voting Classifier for {extracted_string}: {f1:.4f}')

    # Store metrics for this fold
    fold_metrics['Accuracy'].append(accuracy)
    fold_metrics['Precision'].append(precision)
    fold_metrics['Recall'].append(recall)
    fold_metrics['F-measure'].append(f1)
    # Calculate average metrics for this dataset
    avg_metrics = {
        'Accuracy': np.mean(fold_metrics['Accuracy']),
        'Precision': np.mean(fold_metrics['Precision']),
        'Recall': np.mean(fold_metrics['Recall']),
        'F-measure': np.mean(fold_metrics['F-measure'])
    }
    results.append([
        file_path,
        f"{avg_metrics['Accuracy'] * 100:.2f}".replace('.', ','),
        f"{avg_metrics['Precision'] * 100:.2f}".replace('.', ','),
        f"{avg_metrics['Recall'] * 100:.2f}".replace('.', ','),
        f"{avg_metrics['F-measure'] * 100:.2f}".replace('.', ',')
    ])

    # Save results to a CSV file
    output_file = rf"G:\My Drive\AIL303m\ail2-main\BaggingNewResult(36_Tunning) .xlsx"
    # Create a workbook and select the active worksheet
    wb = Workbook()
    ws = wb.active
    # Write the header
    ws.append(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure'])

    # Write the results
    for result in results:
        ws.append(result)

    # Save the workbook
    wb.save(output_file)

Dạ thưa cô, đây là Bagging Models dùng Voting với 52 features và có Hypterparameter Tunning ạ

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
# Đường dẫn tới các file CSV
file_paths_basic = [
    r"G:\My Drive\AIL303m\ail2-main\basicv3\Merged_BC-15_features_v3.csv",
    r"G:\My Drive\AIL303m\ail2-main\basicv3\Merged_Huong_thom-1_features_v3.csv",
    r"G:\My Drive\AIL303m\ail2-main\basicv3\Merged_Nep-87_features_v3.csv",
    r"G:\My Drive\AIL303m\ail2-main\basicv3\Merged_Q-5_modify_features_v3.csv",
    r"G:\My Drive\AIL303m\ail2-main\basicv3\Merged_Thien_uu-8_features_v3.csv",
    r"G:\My Drive\AIL303m\ail2-main\basicv3\Merged_Xi-23_features_v3.csv",
]
results = []
# Suppress specific warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
# Biến để tính tổng độ chính xác
total_accuracy = 0
total_precision = 0
total_recall = 0
total_f1 = 0
# Hàm chính để huấn luyện và đánh giá mô hình
for file_path in file_paths_basic:
    name = file_path
    match = re.search(r'Merged_(.*?)_features', file_path)
    if match:
        extracted_string = match.group(1)
    # Đọc dữ liệu từ file CSV
    print(f"Processing file: {file_path}")
    dataset = pd.read_csv(file_path)
    fold_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F-measure': []
    }
    # Tiền xử lý dữ liệu
    X = dataset.drop(columns=['filename', 'label'], axis=1)
    y = dataset['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    k = 5
    # Create KFold object
    kf = KFold(n_splits=k, shuffle=True, random_state=25)
    # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)
    # Parameter grids
    param_grid_svm = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['rbf', 'linear','poly']
    }

    param_grid_knn = {
        'n_neighbors': range(1, 31),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    }

    param_grid_rf = {
        'n_estimators': [100, 200, 500],
        'max_features': [7,8], 
        'max_depth': [4, 6, 8, 10],
        'criterion': ['gini', 'entropy']
    }

    param_grid_ann = {
        'hidden_layer_sizes': [(50,), (100,), (150,), (100, 100)],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant', 'adaptive']
    }

    param_grid_lr = {
        'C': [0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga'],
        'penalty': ['l2', 'l1'],
        'max_iter': [100, 200, 300]
    }

    # Define the base models
    svm_base = SVC()
    knn_base = KNeighborsClassifier()
    rf_base = RandomForestClassifier(random_state=42)
    ann_base = MLPClassifier(max_iter=1000)
    lr_base = LogisticRegression()

    # GridSearchCV for each model
    gs_svm = GridSearchCV(estimator=svm_base, param_grid=param_grid_svm, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_knn = GridSearchCV(estimator=knn_base, param_grid=param_grid_knn, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_rf = GridSearchCV(estimator=rf_base, param_grid=param_grid_rf, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_ann = GridSearchCV(estimator=ann_base, param_grid=param_grid_ann, cv=kf, verbose=1, scoring='accuracy',error_score='raise')
    gs_lr = GridSearchCV(estimator=lr_base, param_grid=param_grid_lr, cv=kf, verbose=1, scoring='accuracy',error_score='raise')

    # Fit GridSearchCV to the data
    gs_svm.fit(X_train, y_train)
    gs_knn.fit(X_train, y_train)
    gs_rf.fit(X_train, y_train)
    gs_ann.fit(X_train, y_train)
    gs_lr.fit(X_train, y_train)

    # Get the best estimators
    best_svm = gs_svm.best_estimator_
    best_knn = gs_knn.best_estimator_
    best_rf = gs_rf.best_estimator_
    best_ann = gs_ann.best_estimator_
    best_lr = gs_lr.best_estimator_

    print("Best SVM Estimator:", best_svm)
    print("Best KNN Estimator:", best_knn)
    print("Best Random Forest Estimator:", best_rf)
    print("Best ANN Estimator:", best_ann)
    print("Best Logistic Regression Estimator:", best_lr)
    # Định nghĩa các bộ phân loại Bagging
    bagging_svm = BaggingClassifier(best_svm, n_estimators=5, random_state=42, bootstrap=True, max_features=36)
    bagging_ann = BaggingClassifier(best_ann, n_estimators=5, random_state=42, bootstrap=True, max_features=36)
    bagging_RF = BaggingClassifier(best_rf, n_estimators=5, random_state=42, bootstrap=True, max_features=36)
    bagging_knn = BaggingClassifier(best_knn, n_estimators=5, random_state=42, bootstrap=True, max_features=36)
    bagging_lr = BaggingClassifier(best_lr,n_estimators=5,random_state=42,bootstrap=True,max_features=36)
    # Định nghĩa VotingClassifier với phương pháp hard voting
    voting_model = VotingClassifier(estimators=[
        ('svm', bagging_svm),
        ('rf', bagging_RF),
        ('ann', bagging_ann),
        ('lr', bagging_lr),
        ('knn', bagging_knn)
    ], voting='soft')  # Sử dụng soft voting
    # Huấn luyện Voting Classifier
    voting_model.fit(X_train, y_train)
    # Dự đoán trên tập kiểm tra
    y_pred = voting_model.predict(X_test)

    # Đánh giá độ chính xác của Voting Classifier
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f'Accuracy of Voting Classifier for {extracted_string}: {accuracy:.4f}')
    print(f'Precision of Voting Classifier for {extracted_string}: {precision:.4f}')
    print(f'Recall of Voting Classifier for {extracted_string}: {recall:.4f}')
    print(f'F1-score of Voting Classifier for {extracted_string}: {f1:.4f}')

    # Store metrics for this fold
    fold_metrics['Accuracy'].append(accuracy)
    fold_metrics['Precision'].append(precision)
    fold_metrics['Recall'].append(recall)
    fold_metrics['F-measure'].append(f1)
    # Calculate average metrics for this dataset
    avg_metrics = {
        'Accuracy': np.mean(fold_metrics['Accuracy']),
        'Precision': np.mean(fold_metrics['Precision']),
        'Recall': np.mean(fold_metrics['Recall']),
        'F-measure': np.mean(fold_metrics['F-measure'])
    }
    results.append([
        file_path,
        f"{avg_metrics['Accuracy'] * 100:.2f}".replace('.', ','),
        f"{avg_metrics['Precision'] * 100:.2f}".replace('.', ','),
        f"{avg_metrics['Recall'] * 100:.2f}".replace('.', ','),
        f"{avg_metrics['F-measure'] * 100:.2f}".replace('.', ','),
    ])

    # Save results to a CSV file
    output_file = r"G:\My Drive\AIL303m\ail2-main\BaggingNewResult(52_Tuning).xlsx"
    # Create a workbook and select the active worksheet
    wb = Workbook()
    ws = wb.active
    # Write the header
    ws.append(['File', 'Accuracy', 'Precision', 'Recall', 'F-measure'])

    # Write the results
    for result in results:
        ws.append(result)

    # Save the workbook
    wb.save(output_file)